In [193]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn.functional as F
from PIL import Image
import os

In [194]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [195]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(),
    transforms.ToTensor()
])

In [196]:
lr =0.001
batch_size=100
epochs = 15

In [197]:
# Load train and test data
train_dir = r"C:\Users\lizal\Desktop\ML\CatDog\PetImages"
import glob

#train_list = glob.glob(os.path.join(train_dir,'*.jpg')) # LINUX
train_list = []

for path, currentDirectory, files in os.walk(train_dir):
    for file in files:
        train_list.append(os.path.join(path, file))


In [198]:
class dataset(torch.utils.data.Dataset):
    def __init__(self,file_list,transform = None):
        self.file_list=file_list
        self.transform=transform

    def __len__(self):
        self.filelength =len(self.file_list)
        return self.filelength

    def __getitem__(self,idx):
        img_path =self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)

        label = os.path.split(os.path.split(img_path)[-2])[-1]
        if label == 'Dog':
            label = torch.tensor(1)
        elif label == 'Cat':
            label = torch.tensor(0)

        return img_transformed,label

In [199]:
from sklearn.model_selection import train_test_split
train_list,val_list = train_test_split(train_list , test_size =0.2)

In [200]:
train_data = dataset(train_list,transform=transform)
val_data = dataset(val_list,transform=transform)

In [201]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size,shuffle = True)
val_loader = torch.utils.data.DataLoader(dataset = val_data, batch_size = batch_size,shuffle = True)

In [202]:
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn,self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 64,kernel_size=3,padding=1,stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128,kernel_size=3,padding=1,stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)

        )


        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256,kernel_size=3,padding=1,stride=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2)

        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 512,kernel_size=3,padding=1,stride=2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2)

        )

        self.fc1 = nn.Linear(512, 1024)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 2)
        self.relu = nn.ReLU()

    def forward(self,x):
        out =self.layer1(x)
        out =self.layer2(out)
        out =self.layer3(out)
        out =self.layer4(out)
        out =out.view(out.size(0),-1)
        out =self.relu(self.fc1(out))
        out =self.relu(self.fc2(out))
        out =self.dropout(out)
        out =self.fc3(out)
        return out


In [203]:
model = Cnn().to(device)
model.train()
optimizer = optim.Adam(params = model.parameters(),lr =0.001)
criterion = nn.CrossEntropyLoss()

In [204]:
epochs = 15

for epoch in range(epochs):
    epoch_loss =0
    epoch_accuracy = 0

    for data,label in train_loader:
        data= data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output,label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = ((output.argmax(dim=1)==label).float().mean())
        epoch_accuracy += acc/len(train_loader)
        epoch_loss += loss/len(train_loader)

    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch+1, epoch_accuracy,epoch_loss))

    with torch.no_grad():
        epoch_val_accuracy =0
        epoch_val_loss = 0
        for data,label in  val_loader:
            data= data.to(device)

            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output,label)


            acc = ((val_output.argmax(dim=1)==label).float().mean())
            epoch_val_accuracy += acc/len(val_loader)
            epoch_val_loss += val_loss/len(val_loader)
        print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch+1, epoch_val_accuracy,epoch_val_loss))


Epoch : 1, train accuracy : 0.7101794481277466, train loss : 0.5601237416267395
Epoch : 1, val_accuracy : 0.7806000709533691, val_loss : 0.4504290521144867
Epoch : 2, train accuracy : 0.8166294693946838, train loss : 0.4040977656841278
Epoch : 2, val_accuracy : 0.8345999121665955, val_loss : 0.3759148120880127
Epoch : 3, train accuracy : 0.8579844236373901, train loss : 0.3278169333934784
Epoch : 3, val_accuracy : 0.8458001613616943, val_loss : 0.3561974763870239
Epoch : 4, train accuracy : 0.8950397968292236, train loss : 0.25196221470832825
Epoch : 4, val_accuracy : 0.8520000576972961, val_loss : 0.34110599756240845
Epoch : 5, train accuracy : 0.9215458035469055, train loss : 0.1943846195936203
Epoch : 5, val_accuracy : 0.8434000015258789, val_loss : 0.40895459055900574
Epoch : 6, train accuracy : 0.9453941583633423, train loss : 0.1366310566663742
Epoch : 6, val_accuracy : 0.8403998613357544, val_loss : 0.4744042158126831
Epoch : 7, train accuracy : 0.9609969258308411, train loss : 